<a href="https://colab.research.google.com/github/Koga-MD/DL-Tauopathies/blob/main/1_Object_Detection_v5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Use this code when restarting the program. "darknet" folder should be removed before downloading YOLO from Github again.
import shutil
shutil.rmtree("/content/darknet")

In [ ]:
# To mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import cv2
import shutil
import os
from google.colab import output
import threading
import glob
import matplotlib.pyplot as plt
import os.path
from PIL import Image
import sys
from datetime import datetime
import numpy as np
import pandas as pd
import csv
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn

In [ ]:
# Download Darknet from Github
os.chdir("/content")
!git clone https://github.com/AlexeyAB/darknet

# For setting GPU
%cd darknet
!sed -i 's/GPU=0/GPU=1/g' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/g' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/g' Makefile
!sed -i 's/OPENCV=0/OPENCV=1/g' Makefile
!cat Makefile
!make

!mkdir /content/darknet/drive
!mkdir /content/darknet/drive/result
!mkdir /content/darknet/drive/result2
!mkdir /content/darknet/task1
!mkdir /content/darknet/task1/backup
!mkdir /content/darknet/task1/predict
!mkdir /content/darknet/task1/result
!mkdir /content/darknet/task1/test
!mkdir /content/darknet/task1/data
!mkdir /content/darknet/task1/train

!cp /content/darknet/data/voc.names /content/darknet/task1/obj.names
!cp /content/darknet/cfg/yolov3.cfg /content/darknet/task1/yolov3.cfg
!cp "/content/drive/My Drive/task1/process.py" "/content/darknet/process.py"

# Change the number of classes for training.
!sed -i 's/classes= 20/classes= 5/g' cfg/voc.data
!sed -i 's/\/home\/pjreddie\/data\/voc\/train.txt/task1\/train.txt/g' cfg/voc.data
!sed -i 's/\/home\/pjreddie\/data\/voc\/2007_test.txt/task1\/test.txt/g' cfg/voc.data
!sed -i 's/data\/voc.names/task1\/obj.names/g' cfg/voc.data
!sed -i 's/\/home\/pjreddie\/backup\//backup/g' cfg/voc.data

# Change class names
!sed -i 's/aeroplane/NI/g' task1/obj.names
!sed -i 's/bicycle/TA/g' task1/obj.names
!sed -i 's/bird/NP/g' task1/obj.names
!sed -i 's/boat/AP/g' task1/obj.names
!sed -i 's/bottle/CB/g' task1/obj.names
!sed -i '6,21d' task1/obj.names

#Change parameters for training
!sed -i 's/batch=1/#batch=1/g' task1/yolov3.cfg
!sed -i 's/subdivisions=1/#subdivisions=1/g' task1/yolov3.cfg
!sed -i 's/# batch=64/batch=128/g' task1/yolov3.cfg
!sed -i 's/max_batches = 500200/max_batches = 10000/g' task1/yolov3.cfg
!sed -i 's/classes=80/classes=5/g' task1/yolov3.cfg
!sed -i 's/filters=255/filters=30/g' task1/yolov3.cfg
!sed -i 's/# #subdivisions=16/subdivisions=32/g' task1/yolov3.cfg

!cat task1/yolov3.cfg


In [ ]:
# Prepare training dataset
# Copy Training data from Google Drive
!cp "/content/drive/My Drive/task1/data/Data.zip" "/content/darknet/task1/data/Data.zip"

# Unzip folder for training images
os.chdir("/content/darknet/task1/data")
!unzip "/content/darknet/task1/data/Data.zip"
os.remove("/content/darknet/task1/data/Data.zip")

os.chdir("/content/darknet")
!wget https://pjreddie.com/media/files/darknet53.conv.74  # download darknet53.conv.74 from URL to content/darknet

# Split dataset into training and test sets (default = 8:2)
!chmod 755 process.py
!python process.py

In [ ]:
# Train the model
# Automatically clear output and save the log and chart files
 
times = []
for y in range(86400):
    if y %60==0:
      times.append(y)
    else:
      continue
def gfg():
    output.clear()

for x in range(len(times)):
    timer = threading.Timer(times[x],gfg)
    timer.start()

timess = []
for z in range(86400):
    if z %600==0:
      timess.append(z)
    else:
      continue
def asave():
    !cp "/content/darknet/Chartlog.txt" "/content/drive/My Drive/task1/Log.txt"
    !cp "/content/darknet/chart.png" "/content/drive/My Drive/task1/Chart.png" 
    !cp "/content/darknet/backup/yolov3_best.weights" "/content/drive/My Drive/task1/Training-best.weights" 

for x in range(len(timess)):
    timers = threading.Timer(timess[x],asave)
    timers.start()

!./darknet detector train cfg/voc.data task1/yolov3.cfg /content/darknet/darknet53.conv.74 -map >> Chartlog.txt -dont_show

In [ ]:
# Tuned models and image dataset are available at Zenodo (https://zenodo.org/record/5083997)
# Before using the weight, copy the best weight from the Google Drive
!cp "/content/drive/MyDrive/task1/Model1.weights" "/content/darknet/backup/yolov3_best.weights"

# Unzip test files
shutil.rmtree("/content/darknet/task1/test/")
!mkdir /content/darknet/task1/test
!cp "/content/drive/MyDrive/task1/Test.zip" "/content/darknet/task1/test/Test.zip"
os.chdir("/content/darknet/task1/test")

!unzip "/content/darknet/task1/test/Test.zip"
os.remove("/content/darknet/task1/test/Test.zip")

# Rename files in test folder
with open('/content/darknet/task1/CaseList.txt', 'w') as f:
  print("Test-Image", 'Test ID', file=f)

files = sorted(glob.glob('/content/darknet/task1/test/*'))

for i, f in enumerate(files):
    ftitle, fext = os.path.splitext(f)
    os.rename(f, ftitle[:-9] + 'Test' + '{0:03d}'.format(i+1) + fext)
    with open('/content/darknet/task1/CaseList.txt', 'a') as f:
      print(ftitle[-9:], 'Test' + '{0:03d}'.format(i+1), file=f)



In [ ]:
# Run Object detection for test images

with open('/content/darknet/task1/Result.txt', 'w') as f:
      print("Case-ID", "TA", "AP", "CB", "NI", "NP", file=f)

path, dirs, files = next(os.walk("/content/darknet/task1/test"))
fnum = len(files)

for x in range(fnum):
    PATH = "/content/darknet/task1/test/Test"+str(x+1+0).zfill(3)+".jpg"
    fname = str(PATH[-11:-4])
    os.chdir("/content/darknet")
    shutil.rmtree("/content/darknet/task1/predict")
    os.mkdir("/content/darknet/task1/predict")
    shutil.rmtree("/content/darknet/task1/result")
    os.mkdir("/content/darknet/task1/result")
    
    im = Image.open(PATH)
    height = 500
    width = 500
    ncol = im.size[0]//height
    nrow = im.size[1]//width
    ntile = ncol * nrow
    # Spilt tiles from the prediction image
    def ImgSplit(im):
        for h1 in range(nrow):
            for w1 in range(ncol):
                w2 = w1 * height
                h2 = h1 * width
                print(w2, h2, width + w2, height + h2)
                yield im.crop((w2, h2, width + w2, height + h2))

    if __name__ == '__main__':
        # Open image file
        im = Image.open(PATH)
        for number, ig in enumerate(ImgSplit(im), 1):
            ig.save("/content/darknet/task1/predict/test%03d.jpg" % number, "JPEG")

    def display_image(file_path = './predictions.jpg'): 
        fig,ax = plt.subplots()
        ax.tick_params(labelbottom="off",bottom="off")
        ax.tick_params(labelleft="off",left="off")
        ax.set_xticklabels([]) 
        ax.axis('off')

        if os.path.exists(file_path):
          img = cv2.imread(file_path)
          show_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) 
          plt.imshow(show_img)

    NI=[]
    TA=[]
    NP=[]
    AP=[]  
    CB=[]
    RF=[]
    Count=[]

    for i in range(1, ntile + 1):
        path1="/content/darknet/task1/predict/test"+str(i).zfill(3)+".jpg"
        path2="/content/darknet/task1/predict/test.jpg"
        os.rename(path1, path2)
        !./darknet detector test cfg/voc.data /content/darknet/task1/yolov3.cfg /content/darknet/backup/yolov3_best.weights /content/darknet/task1/predict/test.jpg -dont_show -out myfile  > /content/darknet/task1/predict/result.txt
        os.rename(path2, path1)
        

    # Show the result based on result.txt
        with open("/content/darknet/task1/predict/result.txt", 'r') as f:
            lines  = [line.rstrip("\n") for line in f]
            prev_line = ""
            NIsub=[]
            TAsub=[]
            NPsub=[]
            APsub=[]  
            CBsub=[]
            for line in lines:
                args = line.split(' ')
                if args[0]=="NI:" :
                  NI.append(int(args[1][:-1]))
                  NIsub.append(int(args[1][:-1]))
                if args[0]=="TA:" :
                  TA.append(int(args[1][:-1]))
                  TAsub.append(int(args[1][:-1]))
                if args[0]=="NP:" :
                  NP.append(int(args[1][:-1]))
                  NPsub.append(int(args[1][:-1]))
                if args[0]=="AP:" :
                  AP.append(int(args[1][:-1]))
                  APsub.append(int(args[1][:-1]))
                if args[0]=="CB:" :
                  CB.append(int(args[1][:-1]))
                  CBsub.append(int(args[1][:-1]))
        if len(NIsub)>0 or len(TAsub)>0 or len(NPsub)>0 or len(APsub)>0 or len(CBsub)>0:
          Count.append(i)
              
        path3="/content/darknet/predictions.jpg"
        path4="/content/darknet/task1/result/predicted"+str(i).zfill(3)+".jpg"
        os.rename(path3, path4) 

    # Combine all predicted images into one large image.
    # Sort predicted images by name
    list_im = sorted(os.listdir(path='/content/darknet/task1/result'))
    os.chdir("/content/darknet/task1/result/")
    images = [Image.open(x) for x in list_im]
    new_im = Image.new('RGB',(height*ncol, width*nrow))
    x_offset = 0
    for i in range(nrow+1):
        x_offset = 0
        for IM in images[(i-1)*ncol:i*ncol]:
            new_im.paste(IM,(x_offset,height*(i-1)))
            x_offset += IM.size[0]
    new_im.save('Result.jpg')

    # Use Result.txt multiple times
    !cp "/content/darknet/task1/Result.txt" "/content/darknet/task1/result/Result.txt"
    
    with open('Result.txt', 'a') as f:
      print(fname, len(TA), len(AP), len(CB), len(NI), len(NP), file=f)
    
    shutil.rmtree("/content/darknet/drive/result")
    shutil.copytree("/content/darknet/task1/result", "/content/darknet/drive/result")

    # Keep a copy of Result jpeg
    !cp "/content/darknet/drive/result/Result.jpg" "/content/darknet/drive/result2/Result.jpg"  
    old="/content/darknet/drive/result2/Result.jpg"
    new="/content/darknet/drive/result2/res_"+fname+".jpg"
    os.rename(old, new) 

    # Keep a copy of Result text
    !cp "/content/darknet/drive/result/Result.txt" "/content/darknet/drive/result2/Result.txt"  
    old="/content/darknet/drive/result2/Result.txt"
    new="/content/darknet/drive/result2/res_"+fname+".txt"
    os.rename(old, new) 
    !cp "/content/darknet/task1/result/Result.txt" "/content/darknet/task1/Result.txt"

!cp "/content/darknet/task1/CaseList.txt" "/content/darknet/drive/result2/CaseList.txt"
!cp "/content/darknet/task1/Result.txt" "/content/darknet/drive/result2/Result.txt"    

### Convert Result.txt & CaseList.txt to FinalResult.csv
resultfilename = []
with open('/content/darknet/task1/Result.txt', 'r') as f:
    for line in f:
        resultfilename.append(line.split())

caselist = []
with open('/content/darknet/task1/CaseList.txt', 'r') as f:
    for line in f:
        caselist.append(line.split())

# Get 1st row and other keys
resultcolumnname = resultfilename.pop(0)
rc = ["MC_" + i for i in resultcolumnname[1:]]
rc2 = ["CN_" + i for i in resultcolumnname[1:]]
rc3 = ["SF_" + i for i in resultcolumnname[1:]]
rc = rc + rc2 + rc3

# Unify values every 3 list
resultkeyandvalue = resultfilename
resultkeyandvalue = np.array([resultkeyandvalue[i:i + 3] for i in range(0,len(resultkeyandvalue), 3)])
# Get 1st column and change keys' name
resultkeys = list(resultkeyandvalue[:,0,0])
for i in range(len(caselist)):
    sub = {caselist[i][1]: caselist[i][0]}
    resultkeys = [sub.get(x, x).rstrip("DJI") for x in resultkeys]

# Delete picture titles and squeeze values, then change data form to list
resultvalues = np.delete(resultkeyandvalue, 0, axis=2)
resultvalues = resultvalues.reshape(len(resultvalues), -1)
resultvalues = resultvalues.tolist()

# Make dataframe
df = pd.DataFrame(resultvalues,
                  index=resultkeys,
                  columns=rc)
df.index.name = "Case-ID"

df.to_csv("FinalResult.csv", sep=",")

**Decision Tree & Random Forest**

In [ ]:
#####For Manuscript 
# Import our dataset for making Random Forest
df = pd.read_csv("/content/drive/MyDrive/task1/RF-Model1.csv")

pilot = sklearn.utils.Bunch()
pilot['target'] = df['Dx']
pilot['data'] = df.iloc[:, 2:17]
label = df.columns.values
names = label[2:17]
classes = ['AD','CBD','PiD','PSP']

# Random Forest
from sklearn.model_selection import train_test_split
list_train = []
list_test = []
for i in range (30):
  x_train, x_test, t_train, t_test = train_test_split(
      pilot['data'],pilot['target'], random_state=i)

  from sklearn.ensemble import RandomForestClassifier
  RF = RandomForestClassifier(n_estimators=500)
  RF.fit(x_train, t_train)
  Y_pred = RF.predict(x_test)
  RF.score(x_train, t_train)
  Train_score = round(RF.score(x_train, t_train), 3)
  Test_score = round(RF.score(x_test, t_test), 3)
  list_train.append(Train_score)
  list_test.append(Test_score)
  print(i, "Training Score:", Train_score)
  print(i+1, "Test Score:", Test_score)
print("Average Train Score:", sum(list_train)/len(list_train))
print("Average Test Score:", round(sum(list_test)/len(list_test),3))

In [ ]:
## Hold-out dataset 
# Import our hold-out dataset for diagnosing
df2 = pd.read_csv("/content/drive/MyDrive/task1/HO-CP13-Model1.csv")

x_test2 = df2.iloc[:, 2:17]
t_test2 = df2['Dx']
Y_pred = RF.predict(x_test2)
print("Predicted diagnoses are as follows:", Y_pred)
print(Y_pred)
list_train = []
list_test = []
for i in range (30):
  RF = RandomForestClassifier(n_estimators=500)
  RF.fit(x_train, t_train)
  Test_score = round(RF.score(x_test2, t_test2), 3)
  list_test.append(Test_score)
  print("Test", i+1, ": Test Score =", Test_score)
print("Average Test Score:", round(sum(list_test)/len(list_test),3))

In [ ]:
# Import csv file for making diagnoses
df3 = pd.read_csv("/content/darknet/task1/result/FinalResult.csv")

# Import our dataset for making Random Forest
df = pd.read_csv("/content/drive/MyDrive/task1/RF-Model1.csv")

pilot = sklearn.utils.Bunch()
# 'Dx' to 'target'
pilot['target'] = df['Dx']
# Predictor into 'data'
pilot['data'] = df.iloc[:, 2:17]

# Name class
label = df.columns.values
names = label[2:17]
classes = ['AD','CBD','PiD','PSP']

# Random Forest
# Splits the dataset into train and test data
from sklearn.model_selection import train_test_split
list_train = []
list_test = []
for i in range (30):
  x_train, x_test, t_train, t_test = train_test_split(
      pilot['data'],pilot['target'], random_state=i)

  # Random Forest
  from sklearn.ensemble import RandomForestClassifier

  RF = RandomForestClassifier(n_estimators=500)
  RF.fit(x_train, t_train)
  Y_pred = RF.predict(x_test)
  RF.score(x_train, t_train)
  Train_score = round(RF.score(x_train, t_train), 3)
  Test_score = round(RF.score(x_test, t_test), 3)
  list_train.append(Train_score)
  list_test.append(Test_score)

## Predicted diagnoses are given. 
x_test3 = df3.iloc[:, 1:16]
Y_pred = RF.predict(x_test3)
print("Predicted diagnoses are as follows:\n", Y_pred)
